<a href="https://colab.research.google.com/github/jarom-bradshaw/Ml-Projects/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Triple BAM!!!
- Lead: **Scott**
- Model 1: **Jarom**
- Model 2: **Sam**
- Model 3: **Luke**
- Doc Lead: **Tyler**

## MVP By Friday!!!

# Data Info

## If you find a cool way to break down the data please talk about it below.

# One Model to Rule them All

In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

import matplotlib.pyplot as plt
import pandas as pd

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import MinMaxScaler

import pandas as pd

In [ ]:
bike_db = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')
holdout_df = pd.read_csv("https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/biking_holdout_test_mini.csv")
# bike_db.head()
# bike_db.info()
# holdout_df.head()
# holdout_df.info()
bike_db.describe()

,hr,casual,registered,temp_c,feels_like_c,hum,windspeed,weathersit,season,holiday,workingday
count,112475.000000,112475.000000,112475.000000,112475.000000,112475.000000,112475.000000,112475.000000,112475.000000,112475.000000,112475.000000,112475.000000
mean,11.501098,90.434612,249.193625,15.376487,14.659325,0.636624,13.100614,1.405441,2.495799,0.030300,0.684312
std,6.921864,128.655621,258.267544,9.749467,11.428324,0.190328,7.857600,0.683450,1.101152,0.171412,0.464791
min,0.000000,0.000000,0.000000,-14.700000,-24.000000,0.088900,0.000000,1.000000,1.000000,0.000000,0.000000
25%,6.000000,7.000000,48.000000,7.600000,5.400000,0.484100,7.700000,1.000000,2.000000,0.000000,0.000000
50%,12.000000,36.000000,180.000000,16.000000,16.000000,0.640900,12.200000,1.000000,2.000000,0.000000,1.000000
75%,18.000000,122.000000,360.000000,23.500000,23.500000,0.798800,17.500000,2.000000,3.000000,0.000000,1.000000
max,23.000000,1244.000000,1702.000000,40.500000,49.600000,1.000000,69.800000,4.000000,4.000000,1.000000,1.000000


In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

import matplotlib.pyplot as plt
import pandas as pd

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from scipy.stats import zscore

from sklearn.preprocessing import MinMaxScaler

import pandas as pd
bike_db = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')
holdout_df = pd.read_csv("https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/biking_holdout_test_mini.csv")
# bike_db.head()
# bike_db.info()
# holdout_df.head()
# holdout_df.info()

# Luke's data manipulation
bikes = bike_db.rename(columns={'season': 'season_name', 'weathersit': 'weather','temp_c':'temp','feels_like_c':'atemp'})
holdout_df = holdout_df.rename(columns={'season': 'season_name', 'weathersit': 'weather','temp_c':'temp','feels_like_c':'atemp'})

bikes['season_name'] = bikes['season_name'].map({1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'})
bikes = pd.get_dummies(bikes, columns=['season_name'], prefix='', prefix_sep='')
holdout_df['season_name'] = holdout_df['season_name'].map({1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'})
holdout_df = pd.get_dummies(holdout_df, columns=['season_name'], prefix='', prefix_sep='')

bikes['weather'] = bikes['weather'].map({1: 'Clear', 2: 'Mist', 3: 'Light Snow/Rain', 4: 'Heavy Rain/Snow'})
bikes = pd.get_dummies(bikes, columns=['weather'], prefix='', prefix_sep='')
holdout_df['weather'] = holdout_df['weather'].map({1: 'Clear', 2: 'Mist', 3: 'Light Snow/Rain', 4: 'Heavy Rain/Snow'})
holdout_df = pd.get_dummies(holdout_df, columns=['weather'], prefix='', prefix_sep='')

bikes['dteday'] = pd.to_datetime(bikes['dteday'])
holdout_df['dteday'] = pd.to_datetime(holdout_df['dteday'])


bikes['day_of_week'] = bikes['dteday'].dt.dayofweek
holdout_df['day_of_week'] = holdout_df['dteday'].dt.dayofweek


bikes['month'] = bikes['dteday'].dt.month
holdout_df['month'] = holdout_df['dteday'].dt.month


bikes['year'] = bikes['dteday'].dt.year
holdout_df['year'] = holdout_df['dteday'].dt.year


bikes['day_of_month'] = bikes['dteday'].dt.day
holdout_df['day_of_month'] = holdout_df['dteday'].dt.day

bikes['total_bikes'] = bikes['casual'] + bikes['registered']


# Drop top and bottom percentile
# low = bikes['total_bikes'].quantile(0.025)
# high = bikes['total_bikes'].quantile(.975)

# bikes = bikes[(bikes['total_bikes'] >= low) & (bikes['total_bikes'] <= high)]
bikes = bikes.drop(columns=['dteday'])

# z_scores = bikes.apply(zscore)
# bikes = bikes[(z_scores.abs() < 3).all(axis=1)]



tf.config.optimizer.set_jit(True)

X = bikes.drop(columns=['total_bikes','casual','registered'])
# hold_X = holdout_df.drop(columns=['dteday'])

y = bikes['total_bikes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


norm = MinMaxScaler().fit(X_train)
X_train = norm.transform(X_train)
X_test = norm.transform(X_test)

X_train = np.array(X_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)

model = Sequential()
model.add(Dense(512, input_dim=len(X_train[0]), activation='swish', kernel_regularizer=l2(0.01)))
model.add(Dense(512, activation='swish', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(256, activation='swish', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='swish', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='swish', kernel_regularizer=l2(0.01)))
model.add(Dense(1, activation='swish', kernel_regularizer=l2(0.01)))

model.compile(optimizer=Adam(learning_rate=.0005), loss='mean_squared_error', metrics=['mse'])

early_stop = keras.callbacks.EarlyStopping(monitor='val_mse', patience=15, mode='min', restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=1000, validation_split=0.35, batch_size=128, shuffle=True, callbacks=[early_stop])

hist = pd.DataFrame(history.history)

model.evaluate(X_test, y_test, verbose=1)

y_pred = model.predict(X_test)

y_pred = np.round(y_pred).astype(int)
y_pred = np.maximum(y_pred, 0)

print(f"R^2: {r2_score(y_test, y_pred)}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/1000
400/400 ━━━━━━━━━━━━━━━━━━━━ 15s 18ms/step - loss: 130542.1641 - mse: 130529.2969 - val_loss: 72819.1484 - val_mse: 72804.7188
Epoch 2/1000
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 73056.2344 - mse: 73041.6875 - val_loss: 70726.6484 - val_mse: 70711.2578
Epoch 3/1000
400/400 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 69638.2188 - mse: 69621.9688 - val_loss: 48852.4766 - val_mse: 48832.0117
Epoch 4/1000
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 50147.6133 - mse: 50125.8984 - val_loss: 39418.2109 - val_mse: 39393.5664
Epoch 5/1000
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 41539.4062 - mse: 41513.8516 - val_loss: 31005.6934 - val_mse: 30977.7344
Epoch 6/1000
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 35309.7109 - mse: 35281.1328 - val_loss: 27191.8496 - val_mse: 27161.4629
Epoch 7/1000
400/400 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 31546.8633 - mse: 31515.9531 - val_loss: 24315.3008 - val_mse: 24282.8262
Epoch 8/1000
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s

In [ ]:
# holdout_df = pd.read_csv("https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/biking_holdout_test_mini.csv")
holdout_df = pd.read_csv("https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes_december.csv")

holdout_df = holdout_df.rename(columns={'season': 'season_name', 'weathersit': 'weather','temp_c':'temp','feels_like_c':'atemp'})

holdout_df['season_name'] = holdout_df['season_name'].map({1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'})
holdout_df = pd.get_dummies(holdout_df, columns=['season_name'], prefix='', prefix_sep='')

holdout_df['weather'] = holdout_df['weather'].map({1: 'Clear', 2: 'Mist', 3: 'Light Snow/Rain', 4: 'Heavy Rain/Snow'})
holdout_df = pd.get_dummies(holdout_df, columns=['weather'], prefix='', prefix_sep='')

holdout_df['dteday'] = pd.to_datetime(holdout_df['dteday'])

holdout_df['day_of_week'] = holdout_df['dteday'].dt.dayofweek

holdout_df['month'] = holdout_df['dteday'].dt.month

holdout_df['year'] = holdout_df['dteday'].dt.year

holdout_df['day_of_month'] = holdout_df['dteday'].dt.day

expected_seasons = ['Winter', 'Spring', 'Summer', 'Fall']
expected_weather = ['Clear', 'Mist', 'Light Snow/Rain', 'Heavy Rain/Snow']


for season in expected_seasons:
    column_name = f"{season}"
    if column_name not in holdout_df.columns:
        holdout_df[column_name] = 0

# Ensure all weather columns exist
for weather in expected_weather:
    column_name = f"{weather}"
    if column_name not in holdout_df.columns:
        holdout_df[column_name] = 0


hold_X = holdout_df.drop(columns=['dteday'])
hold_X = hold_X.reindex(columns=X.columns, fill_value=0)

# norm = MinMaxScaler().fit(hold_X)
hold_X = norm.transform(hold_X)
hold_X = np.array(hold_X, dtype=np.float32)


hold_pred = model.predict(hold_X)


pd.DataFrame(hold_pred, columns=['prediction']).to_csv('team4-module4-predictions.csv', index=False)



46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


# Charts and Graphics